*Construir un programa en el cual sólo definiremos la función y pueda aplicar los métodos de bisección, falsa posición, secante y en su caso el método de Müller si es un polinomio. Únicamente se ingresará un intervalo y el error, y regresará una aproximación de la raíz.*

**Procedimiento:**

Para realizar este programa, necesitaremos declarar las funciones base para cada uno de los métodos:

*   Método de bisección
*   Falsa posición
*   Método de Secante
*   Método de Müller

Pero antes que eso, necesitaremos importar ciertas librerías que nos ayudarán a abstraer las funciones del usuario y brindarnos algunas bondades extra del lenguaje.




In [1]:
#Importamos librerías

#Redondeo por debajo
from math import ceil
#Funciones matemáticas comúnes
from numpy import pi, sin, cos, tan, log, log10, exp, sqrt

In [2]:
#Método de bisección iterativo
def metBiseccion(a, b, fx, etol, error, k, panterior):
    if fx(a)*fx(b)>0:
      print('No existen raices de la funcion objetivo en el intervalo dado \n')
      return "Error"
    
    # En este caso se ha encontrado la raíz y esta puede estar en a o en b.
    elif fx(a)*fx(b)==0:
        if fx(a)==0:
            return a
        else:
            return b
    
    elif error > etol and k < 100:
        # Calculamos el punto medio
        pmedio = a + 0.5*(b-a)
        k = k + 1
        if fx(a)*fx(pmedio)<0:
            # Redefinimos b = punto medio
            b = pmedio
        elif fx(pmedio)*fx(b)<0:
            # Redefinimos a = punto medio
            a = pmedio
        else:
            # Se ha encontrada la raíz y regresamos la raíz
            return pmedio
        error = abs((pmedio - panterior)/panterior)
        # Llamamos a la función
        return metBiseccion(a, b, fx, etol, error, k, pmedio)
    else:
        # Devolvemos el último valor, después del max
        return panterior

Ahora el método de Falsa posición, cambiando el punto medio por una ecuación de la recta, brindándonos una mejor aproximación cada iteración:

$$c=b-f\left(b\right)\left[\frac{b-a}{f\left(b\right)-f\left(a\right)}\right]$$

In [3]:
#Método Falsa posición
def metFalsaPosicion(a, b, fx, etol, error, k, panterior):
    if fx(a)*fx(b)>0:
      print('No existen raices de la funcion objetivo en el intervalo dado \n')
      return "Error"
    
    # En este caso se ha encontrado la raíz y esta puede estar en a o en b.
    elif fx(a)*fx(b)==0:
        if fx(a)==0:
            return a
        else:
            return b

    elif error > etol and k < 100:
        # Usamos al modelo matemático del método de falsa posición
        pmedio = b-fx(b)*((b-a)/(fx(b)-fx(a)))
        k = k + 1
        # Redifinimos a o b
        if fx(a)*fx(pmedio)<0:
            b = pmedio
        elif fx(pmedio)*fx(b)<0:
            a = pmedio
        else:
        # Encontramos la raíz, la devolvemos
            return pmedio
        error = abs((pmedio - panterior)/panterior)
        # Llamamos a la función
        return metFalsaPosicion(a, b, fx, etol, error, k, pmedio)
    else:
        #Devolvemos el último valor
        return panterior

Para el método Secante se necesitan 2 aproximaciones iniciales, y con ellas  ocuparemos este método dado por:

$$p_n = p_{n-1} - \frac{f\left(p_{n-1}\right)\left(p_{n-1}-p_{n-2}\right)}{f\left(p_{n-1}\right)-f\left(p_{n-2}\right)}$$

In [4]:
#Método Secante
def metSecante(p0, p1, fx, etol, error, k):
    if fx(p1)==0:
        return p1
    
    # Dado que existe una raíz en el intervalo comenzaremos el proceso iterativo
    elif error > etol and k < 100:
        # Calculamos p_n con el modelo matemático recursivo
        psecante = p1 - fx(p1)*(p1-p0)/(fx(p1)-fx(p0))
        k = k + 1
        error = abs(p1 - psecante) + abs(fx(psecante)) # Error de aproximaciones
        p0 = p1
        p1 = psecante
        #Llamamos a la función
        return metSecante(p0, p1, fx, etol, error, k)
    else:
        # Devolvemos el último valor
        return p1

En caso de que la función de entrada sea un polinomio, ocuparemos el método de Müller, que se basa en acoplar el polinomio de grado $n$ a uno cuadrático, con el fin de ocupar la famosa fórmula general, finalmente el método de Müller se describe de la siguiente manera:

$$p_{n+1} = p_n - \frac{2c}{b + sgn\left(b\right)\sqrt{b^2 - 4ac}}$$

In [5]:
# Definimos la función signo
def sgn(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

# Método Müller
def metMuller(p0, p1, p2, fx, etol, error, k):
    if fx(p2)==0:
        return p2
    
    # Dado que existe una raíz en el intervalo comenzaremos el proceso iterativo
    elif error > etol and k < 100:
      # Actualizamos Coeficientes
        # Calculamos el punto medio
        f1 = fx(p1) - fx(p2)
        f2 = fx(p0) - fx(p2)
        x1 = p0 - p1
        x2 = p1 - p2
        x3 = p0 - p2
        
        # Calculamos el coeficiente a,b y c
        a = (f2 * x2 - f1 * x3) / (x3 * x2 * x1)
        b = (f1 * x3 * x3 - f2 * x2 * x2) / (x3 * x2 * x1)
        c = fx(p2)

        k = k + 1
        
        p0 = p1
        p1 = p2

        # Calculamos p_n+1 con los datos anteriores
        p2 = p1 - (2*c)/(b+(sgn(b))*sqrt(b**2 -4*a*c + 0j))

        error = abs(fx(p2))
        # Llamamos a la función
        return metMuller(p0, p1, p2, fx, etol, error, k)
    else:
        # Devolvemos el último valor
        return p2

In [6]:
# Función para unificar los métodos
def unionMetodos(p1, p2, errorT, fx, esPoli):
    print("Raíz de la función con error de", errorT, "en el intervalo [",p1,", ",p2,"]")
    print("Bisección: ", metBiseccion(p1, p2, fx, errorT, 1000, 0, p2))
    print("Falsa Posición: ", metFalsaPosicion(p1, p2, fx, errorT, 1000, 0, p2))
    print("Secante: ", metSecante(p1, p2, fx, errorT, 1000, 0))
    # En caso de polinomio
    if esPoli:
        print("Müller: ", metMuller(p1, p2, p1+p2, fx, errorT, 1000, 0))

    print("\n")

Ahora usaremos una demostración para utilizar los métodos numéricos anteriormente implementados de manera recursiva, a una función no polinómica y polinómica:

$$f_{d1}(x) = 2-xe^x\\
f_{d2}(x) =3 x^5+2 x^2+x/2$$

Con un error en ambas de $1\times10^{-10}$ y dentro de los intervalos $[0, 1]$ y $[-1, -0.5]$ respectivamente.

In [7]:
def f_d1(x):
    return 2-x*exp(x)

def f_d2(x):
    return 3*x**5+2*x**2+x/2

In [8]:
p11 = 0
p12 = 1
p21 = -1
p22 = -0.5
errorT = 10**-10

In [9]:
unionMetodos(p11, p12, errorT, f_d1, False)
unionMetodos(p21, p22, errorT, f_d2, True)

Raíz de la función con error de 1e-10 en el intervalo [ 0 ,  1 ]
Bisección:  0.8526055020629428
Falsa Posición:  0.8526055020108265
Secante:  0.8526055020137255


Raíz de la función con error de 1e-10 en el intervalo [ -1 ,  -0.5 ]
Bisección:  -0.7657499688793905
Falsa Posición:  -0.7657499688609568
Secante:  -0.2564921337892874
Müller:  (-0.7657499689282056-1.8635503552593144e-18j)




Teniendo para la primera función resultados similares en la raíz hasta el $1\times10^{-10}$ y para la segunda función, que es polinómica, 3 raíces diferentes, teniendo los mismos resultados para el método de Bisección y Falsa Posición.
Ahora dejaremos el siguiente apartado para que el usuario final utilice los métodos para una función cualquiera:

In [ ]:
# Inserte el intervalo y el error tolerable para su función
p1 =
p2 =
errorT =

In [ ]:
# Coloque su función seguido del return
def fx(x):
    return

In [ ]:
# Función para llamar a los métodos
# Ponga True si es polinomio, si no, False
unionMetodos(p1, p2, errorT, fx, True)